In [1]:
%sql
-- setting database in session
use soccer_db

In [2]:
%sql
-- taking a peak at all goals
select
  *
from game_events as ge
where
  ge.is_goal = 1
limit 5

id_odsp,id_event,sort_order,time,event_type,event_type_str,event_type2,event_type2_str,side,side_str,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_place_str,shot_outcome,shot_outcome_str,is_goal,location,location_str,bodypart,bodypart_str,assist_method,assist_method_str,situation,situation_str,time_bin,country_code
AcBjeS4n/,AcBjeS4n53,53,45,1,Attempt,12,Key Pass,1,Home,Udinese,Parma,kwadwo asamoah,antonio di natale,null,null,3,Bottom left corner,1,On target,1,3,Centre of the box,2,Left foot,3,Headed pass,1,Open play,4.0,ITA
AcBjeS4n/,AcBjeS4n64,64,57,1,Attempt,12,Key Pass,1,Home,Udinese,Parma,antonio di natale,pablo armero,null,null,3,Bottom left corner,1,On target,1,3,Centre of the box,1,Right foot,2,Cross,1,Open play,5.0,ITA
AcBjeS4n/,AcBjeS4n91,91,85,1,Attempt,99,NA,2,Away,Parma,Udinese,alessandro lucarelli,null,null,null,4,Bottom right corner,1,On target,1,3,Centre of the box,2,Left foot,0,None,3,Corner,8.0,ITA
AcBjeS4n/,AcBjeS4n99,99,90,1,Attempt,99,NA,1,Home,Udinese,Parma,kwadwo asamoah,null,null,null,3,Bottom left corner,1,On target,1,3,Centre of the box,2,Left foot,0,None,1,Open play,9.0,ITA
d8tAEYk1/,d8tAEYk139,39,31,1,Attempt,99,NA,1,Home,AC Milan,Fiorentina,zlatan ibrahimovic,null,null,null,3,Bottom left corner,1,On target,1,14,Penalty spot,1,Right foot,0,None,2,Set piece,2.0,ITA


In [3]:
%sql
-- distribution of goals by shot place
select
  case
    when ge.shot_place_str = 'NA' then 'Unknown'
    else ge.shot_place_str
  end as shot_place,
  count(*) as total_goals
from game_events as ge
where
  ge.is_goal = 1
group by
  ge.shot_place_str
order by
  count(*) desc

shot_place,total_goals
Bottom left corner,7212
Bottom right corner,6932
Centre of the goal,4446
Top right corner,2157
Top left corner,2023
Unknown,1676


In [4]:
%sql
-- distribution of goals by country
select
  ge.country_code as country_code,
  count(*) as total_goals
from game_events as ge
where
  ge.is_goal = 1
group by
  ge.country_code
order by
  count(*) desc

country_code,total_goals
ESP,5583
ITA,5491
FRA,5199
DEU,4621
GBR,3552


In [5]:
%sql
-- top 3 goal locations per shot in Spanish league
select
  sq2.location_str as location_str,
  sq2.shot_place_str as shot_place_str,
  sq2.total_goals as total_goals
from (
  select
    sq1.*,
    rank() over(partition by shot_place_str order by total_goals desc) as goals_rank
  from (
    select
      ge.location_str as location_str,
      ge.shot_place_str as shot_place_str,
      count(*) as total_goals
    from game_events as ge
    where
      ge.is_goal = 1
      and ge.country_code = 'ESP'
      and ge.location_str != 'NA'
      and ge.shot_place_str != 'NA'
    group by
      ge.location_str,
      ge.shot_place_str
      ) as sq1
    ) as sq2
where
  sq2.goals_rank <= 3
order by
  sq2.total_goals desc

location_str,shot_place_str,total_goals
Centre of the box,Bottom left corner,832
Centre of the box,Bottom right corner,813
Centre of the box,Centre of the goal,429
Very close range,Centre of the goal,275
Very close range,Bottom left corner,219
Centre of the box,Top right corner,178
Very close range,Bottom right corner,174
Centre of the box,Top left corner,172
Outside the box,Bottom left corner,166
Outside the box,Bottom right corner,163


In [6]:
%sql
-- goals per time bun per country
select
  ge.country_code as country_code,
  ge.time_bin as time_bin,
  count(*) as total_goals
from game_events as ge
where
  is_goal = 1
group by
  ge.country_code,
  ge.time_bin
order by
  ge.country_code,
  ge.time_bin

country_code,time_bin,total_goals
DEU,0.0,418
DEU,1.0,466
DEU,2.0,521
DEU,3.0,417
DEU,4.0,461
DEU,5.0,496
DEU,6.0,486
DEU,7.0,411
DEU,8.0,507
DEU,9.0,438
